# Solutions to exercises in the `ray-core` Lessons

First, import everything we'll need and start Ray:

In [1]:
import ray, time, sys
import numpy as np
sys.path.append('../..')
from util.printing import pnd, pd

In [2]:
ray.init(ignore_reinit_error=True)

2020-04-07 08:02:48,921	INFO resource_spec.py:204 -- Starting Ray with 4.44 GiB memory available for workers and up to 2.23 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-07 08:02:49,274	INFO services.py:1146 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:26771',
 'object_store_address': '/tmp/ray/session_2020-04-07_08-02-48_912726_63557/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-07_08-02-48_912726_63557/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-04-07_08-02-48_912726_63557'}

## Exercise 1 in 02-TaskParallelism-Part1

You were asked to convert the regular Python code to Ray code. Here are the three cells appropriately modified.

First, we need the appropriate imports and `ray.init()`.

In [3]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

In [4]:
start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = ray.get(ids)
duration = time.time() - start

In [5]:
assert squares == [0, 1, 4, 9]
# should fail until the code modifications are made:
assert duration < 4.1, f'duration = {duration}' 

## Exercise 2 in 03-TaskParallelism-Part2

You were asked to use `ray.wait()` with a shorter timeout, `2.5` seconds. First we need to redefine in this notebook the remote functions we used in that lesson:

In [10]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

In [12]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=2.5)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.021)
0: []
10: [ 3.95318532  1.95703209 -1.77142106  0.48401556 -3.66004368  1.65002282
  3.51581126 -0.62342501 -1.43977852 -0.19810529]
Returned   1 completed tasks. (elapsed time:  5.533)
20: [ 1.38744086 -1.84890282  1.76892294  0.88618333 -0.45441002 -0.74875324
 -0.68359449  1.35974112 -2.72374048 -1.6864953   0.17671172  1.99400488
  1.53733697 -1.47867413 -0.51142734 -2.19978562 -3.49982821  1.07790542
 -0.38826834  1.69827754]
Returned   2 completed tasks. (elapsed time:  8.014)
30: [ 1.13530849 -4.07447539  0.30959073  1.30257747  0.95060131 -3.05474103
  2.05413706  3.70526873 -1.9819779   0.52198044 -2.42485218  2.79255394
  0.244146    4.65404811  2.62351162  1.31775663  3.4565208   2.13435246
  2.05918341 -0.40861993 -1.87624159 -0.52302686 -0.45058515  0.73924722
  1.04711594 -0.14052396  2.60024071 -0.51780442 -5.92163164 -0.94569481]
40: [ 0.75586819 -0.23227869 -1.09352479 -0.63175279  1.81580443  1.00967837
  2.73164613

For a timeout of `2.5` seconds, the second call to `ray.wait()` times out before two tasks finish, so it only returns one completed task. Why did the third and last iteration not time out? (That is, they both successfully returned two items.) It's because all the tasks were running in parallel so they had time to finish. If you use a shorter timeout, you'll see more time outs, where zero or one items are returned. 

Try `1.5` seconds, where all but one iteration times out and returns one item. The first iteration returns two items.
Try `0.5` seconds, where you'll get several iterations that time out and return zero items, while all the other iterations time out and return one item.

## Exercise 3 in 03-TaskParallelism-Part2

You were asked to convert the code to use Ray, especially `ray.wait()`.

In [13]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = []
waiting_ids = ids
while len(waiting_ids) > 0:
    finished_ids, waiting_ids = ray.wait(waiting_ids)  # We just assign the second list to waiting_ids...
    squares.extend(ray.get(finished_ids))
duration = time.time() - start

In [14]:
assert squares == [0, 1, 4, 9]
assert duration < 4.1, f'duration = {duration}' 